<center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective">Objective</h3>
<br><strong style="color: #000000;"><em>Find unwanted/unused healthchecks in AWS Route53 and delete them.</em></strong></div>
</center><center>
<h2 id="Change-AWS-Route53-TTL-value"><strong><u>Delete Unused AWS Route53 Healthchecks</u></strong></h2>
</center>
<h1 id="Steps-Overview">Steps Overview</h1>
<p>1)<a href="#1" target="_self" rel="noopener"> Get unused Route53 healthchecks</a><br>2)<a href="#2" target="_self" rel="noopener"> Delete the healthchecks</a></p>

<h3 id="Get-TTLs-under-X-hours"><a id="1" target="_self" rel="nofollow"></a>Get unused Route53 Healthchecks</h3>
<p>Using unSkript's Get Route53 Unused Healthchecks , we will find the healthcheck IDs that are not being used by any record set to monitor their health.</p>
<blockquote>
<p>This action takes the following parameters: <code>hosted_zone_id(Optional)</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>unused_health_checks</code></p>
</blockquote>

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Optional, Tuple
import pprint
from unskript.connectors.aws import aws_get_paginator


from beartype import beartype
@beartype
def aws_get_unused_route53_health_checks_printer(output):
    if output is None:
        return

    pprint.pprint(output)


@beartype
def aws_get_unused_route53_health_checks(handle, hosted_zone_id: str = "") -> Tuple:
    """aws_get_unused_route53_health_checks Returns a list of unused Route 53 health checks.

        :type hosted_zone_id: string
        :param hosted_zone_id: Optional. Used to filter the health checks for a specific hosted zone.

        :rtype: A tuple containing a list of dicts with information about the unused health checks.
    """
    result = []
    try:
        route_client = handle.client('route53')
        health_checks = aws_get_paginator(route_client, "list_health_checks", "HealthChecks")
        if hosted_zone_id:
            hosted_zones = [{'Id': hosted_zone_id}]
        else:
            hosted_zones = aws_get_paginator(route_client, "list_hosted_zones", "HostedZones")
        used_health_check_ids = set()
        for zone in hosted_zones:
            record_sets = aws_get_paginator(route_client, "list_resource_record_sets", "ResourceRecordSets", HostedZoneId=zone['Id'])
            for record_set in record_sets:
                if 'HealthCheckId' in record_set:
                    used_health_check_ids.add(record_set['HealthCheckId'])
        for hc in health_checks:
            if hc['Id'] not in used_health_check_ids:
                result.append(hc['Id'])
    except Exception as e:
        raise e

    if len(result) != 0:
        return (False, result)
    else:
        return (True, None)



task = Task(Workflow())
task.configure(outputName="unused_health_checks")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_unused_route53_health_checks, lego_printer=aws_get_unused_route53_health_checks_printer, hdl=hdl, args=args)

<h3 id="Create-List-of-Lower-TTL-records">Create List of Unused Healthchecks</h3>
<p>This action filters the output from Step 1 to get the non empty values</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>all_unused_health_checks</code></p>
</blockquote>

In [ ]:

all_unused_health_checks = []
dummy = []
try:
    for res in range(len(unused_health_checks)):
        if unused_health_checks[0]==False:
            if len(unused_health_checks[1])!=0:
                    all_unused_health_checks = unused_health_checks[1]
except Exception as e:
    all_unused_health_checks = health_check_ids
print(all_unused_health_checks)
task.configure(outputName="all_unused_health_checks")

<h3 id="Change-TTL-Value"><a id="2" target="_self" rel="nofollow"></a>Delete Route53 Healthcheck</h3>
<p>This action deletes the Route53 healthcheck found in Step 1.</p>
<blockquote>
<p>This action takes the following parameters:&nbsp;<code>health_check_id</code></p>
</blockquote>

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Dict
import pprint


from beartype import beartype
@beartype
def aws_delete_route53_health_check_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_delete_route53_health_check(handle, health_check_id: str) -> Dict:
    """aws_delete_route53_health_check Deletes a Route 53 Health Check.

        :type health_check_id: string
        :param health_check_id: The ID of the Health Check to delete.

        :rtype: dict of health check information.
    """
    try:
        route_client = handle.client('route53')
        response = route_client.delete_health_check(HealthCheckId=health_check_id)
        return response
    except Exception as e:
        raise Exception(e)


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "health_check_id": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "all_unused_health_checks",
    "iter_parameter": "health_check_id"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(all_unused_health_checks)!=0",
    "condition_result": true
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_delete_route53_health_check, lego_printer=aws_delete_route53_health_check_printer, hdl=hdl, args=args)

<h3 id="Conclusion&para;">Conclusion</h3>
<p>In this Runbook, we were able delete unused healtcheck ID's which will help in saving your AWS costs. To view the full platform capabilities of unSkript please visit&nbsp;<a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>